## Visualization of supervised and supervised model side by side

In [1]:
# imports
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tf_dataset.tf_dataset import get_tf_dataset
from model.models import get_model


2024-03-03 17:28:49.371289: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-03 17:28:49.397583: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 17:28:49.397613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 17:28:49.398211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 17:28:49.402452: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# test dataset
root = '/home/ji/Dropbox/Robotics/CMSC733/Project1/Phase2/Data'
train_path = root+"/Train"
val_path = root+"/Val"
test_path = root+"/Test"

train_ds = get_tf_dataset(path=train_path,
                    batch_size=8,
                    mode="unsupervised")

val_ds = get_tf_dataset(path=val_path,
                    batch_size=8,
                    mode="unsupervised")

test_ds = get_tf_dataset(path=test_path,
                    batch_size=8,
                    mode="unsupervised")

2024-03-03 17:28:56.910886: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-03 17:28:56.937357: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-03 17:28:56.941106: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# load models
model_s = get_model(mode="supervised")
model_s.load_weights("./chkpt/mdl_supervised_rho32")

model_us = get_model(mode="unsupervised")
model_us.load_weights("./chkpt/mdl_unsupervised_rho32")

new init
new init


2024-03-03 17:29:30.316688: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-03 17:29:30.687744: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x8dae230


checking nan


In [4]:
from tqdm import tqdm
def Mean_Corner_error(corners1, corners2):
    assert corners1.shape == corners2.shape
    distances = np.sqrt(np.sum(np.square(corners1 - corners2), axis=1))
    mean_error = np.mean(distances)
    return mean_error

def calculate_metric(ds, nimg):
    corner_dist_s = []
    corner_dist_us = []
    B = 8
    # go through all test images

    for i in tqdm(range(int(nimg/B))):
        # retrieve a sample batch
        sample_input, sample_output = next(iter(ds))
        im_crop1, im_crop2, im_ori, upper_left_coord = sample_input
        im_warp, h4pt = sample_output
        h4pt = h4pt.numpy().reshape((-1,4,2))

        h4pt_s = model_s([im_crop1,im_crop2])
        h4pt_s = (np.round(h4pt_s.numpy())).reshape((-1,4,2))

        model_out_us = model_us(sample_input)
        im_warp_pred_us, h4pt_us = model_out_us
        im_warp_pred_us = np.round(im_warp_pred_us.numpy()*255)
        h4pt_us = np.round(h4pt_us.numpy()).reshape((-1,4,2))

        for b in range(B):
            corner_dist_s.append( Mean_Corner_error(np.squeeze(h4pt[b,:,:]),
                                                np.squeeze(h4pt_s[b,:,:]))
                                )
            
            corner_dist_us.append( Mean_Corner_error(np.squeeze(h4pt[b,:,:]),
                                                np.squeeze(h4pt_us[b,:,:]))
                                )
        
    corner_dist_s = np.array(corner_dist_s)
    corner_dist_us = np.array(corner_dist_us)

    return corner_dist_s, corner_dist_us

In [5]:
cd_s_train, cd_us_train = calculate_metric(train_ds, nimg=5000)
cd_s_val, cd_us_val = calculate_metric(val_ds, nimg=1000)
cd_s_test, cd_us_test = calculate_metric(test_ds, nimg=1000)

  0%|          | 0/625 [00:00<?, ?it/s]2024-03-03 17:29:36.979998: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-03 17:29:37.041726: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
100%|██████████| 125/125 [00:27<00:00,  4.57it/s]


In [6]:
def print_result(corner_dist_s, corner_dist_us):
    print( (f"\tsupervised model error: mean {np.mean(corner_dist_s):.3f}, "
            f"std {np.std(corner_dist_s):.3f}")
        )
    print( (f"\tunsupervised model error: mean {np.mean(corner_dist_us):.3f}, "
            f"std {np.std(corner_dist_us):.3f}")
     )

print("Training set:")
print_result(cd_s_train, cd_us_train)

print("Validation set:")
print_result(cd_s_val, cd_us_val)

print("Test set:")
print_result(cd_s_test, cd_us_test)

Training set:
	supervised model error: mean 5.152, std 2.446
	unsupervised model error: mean 10.204, std 4.740
Validation set:
	supervised model error: mean 5.167, std 2.442
	unsupervised model error: mean 9.989, std 4.915
Test set:
	supervised model error: mean 5.232, std 2.618
	unsupervised model error: mean 10.206, std 4.850
